# Week 4 Lab: Linear Regression for Real Estate Price Prediction

## Business Scenario

You've been hired as a data scientist by **Sunshine Realty**, a real estate company operating in a growing metropolitan area. The company wants to:
1. **Accurately price properties** for sale to remain competitive
2. **Identify key factors** that drive property values
3. **Predict future property values** for investment decisions

Your task is to build a linear regression model that can predict house prices based on various features like size, location, age, and amenities.

## Learning Objectives
By completing this lab, you will:
- Understand the ML perspective vs. statistical perspective on linear regression
- Implement train/test/validation splits
- Recognize and handle overfitting vs. underfitting
- Apply feature engineering techniques
- Evaluate models using multiple metrics
- Use regularization to improve model performance
- Create production-ready pipelines

## Part 1: Setup and Data Generation

First, let's import necessary libraries and generate synthetic real estate data that mimics real-world patterns.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Setup complete!")

### Generate Synthetic Real Estate Data

We'll create realistic property data with the following features:
- **Square footage**: Size of the property
- **Bedrooms**: Number of bedrooms
- **Bathrooms**: Number of bathrooms
- **Age**: Age of the property in years
- **Distance to city center**: In miles
- **Crime rate**: Area crime index (0-100)
- **School rating**: Local school quality (1-10)
- **Has pool**: Binary feature
- **Has garage**: Binary feature

The price will be a function of these features with some non-linear relationships and noise.

In [ ]:
def generate_real_estate_data(n_samples=1000):
    """
    Generate synthetic real estate data with realistic relationships.
    """
    np.random.seed(42)
    
    # Generate features
    sqft = np.random.normal(2000, 600, n_samples)
    sqft = np.clip(sqft, 500, 5000)  # Realistic bounds
    
    bedrooms = np.random.choice([1, 2, 3, 4, 5], n_samples, p=[0.1, 0.25, 0.35, 0.25, 0.05])
    bathrooms = bedrooms + np.random.choice([-1, 0, 1], n_samples, p=[0.3, 0.5, 0.2])
    bathrooms = np.clip(bathrooms, 1, 4)
    
    age = np.random.exponential(15, n_samples)
    age = np.clip(age, 0, 50)
    
    distance_to_city = np.random.exponential(10, n_samples)
    distance_to_city = np.clip(distance_to_city, 1, 40)
    
    crime_rate = np.random.beta(2, 5, n_samples) * 100  # Skewed toward lower crime
    
    school_rating = np.random.normal(7, 1.5, n_samples)
    school_rating = np.clip(school_rating, 1, 10)
    
    has_pool = np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
    has_garage = np.random.choice([0, 1], n_samples, p=[0.3, 0.7])
    
    # Generate price with realistic relationships
    price = (
        150 * sqft  # Base price per sqft
        + 20000 * bedrooms  # Value per bedroom
        + 15000 * bathrooms  # Value per bathroom
        - 2000 * age  # Depreciation
        - 5000 * distance_to_city  # Distance penalty
        - 1000 * crime_rate  # Crime penalty
        + 10000 * school_rating  # School quality premium
        + 25000 * has_pool  # Pool premium
        + 15000 * has_garage  # Garage premium
        + 50000  # Base price
    )
    
    # Add non-linear effects
    price += 0.5 * sqft * bedrooms  # Interaction: larger bedrooms in larger houses worth more
    price *= (1 - 0.01 * age) ** 2  # Non-linear depreciation
    
    # Add noise
    noise = np.random.normal(0, 30000, n_samples)
    price = price + noise
    price = np.clip(price, 50000, None)  # Minimum price
    
    # Create DataFrame
    data = pd.DataFrame({
        'sqft': sqft,
        'bedrooms': bedrooms,
        'bathrooms': bathrooms,
        'age': age,
        'distance_to_city': distance_to_city,
        'crime_rate': crime_rate,
        'school_rating': school_rating,
        'has_pool': has_pool,
        'has_garage': has_garage,
        'price': price
    })
    
    return data

# Generate the dataset
df = generate_real_estate_data(1000)
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

## Part 2: Exploratory Data Analysis

Before building models, let's understand our data through visualization and statistics.

### Exercise 2.1: Basic Statistics
**Task**: Display summary statistics and check for any data quality issues.

In [ ]:
# TODO: Display basic statistics about the dataset
# Hint: Use describe() method and info() method

print("Summary Statistics:")
# YOUR CODE HERE: Display summary statistics
______

print("\nData Types and Missing Values:")
# YOUR CODE HERE: Display data types and check for missing values
______

### Exercise 2.2: Visualize Relationships
**Task**: Create visualizations to understand the relationship between features and price.

In [ ]:
# Create a figure with subplots for key relationships
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# TODO: Create scatter plots for continuous variables vs price
# Plot 1: Square footage vs Price
axes[0, 0].scatter(df['sqft'], df['price'], alpha=0.5)
axes[0, 0].set_xlabel('Square Footage')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].set_title('Price vs Square Footage')

# YOUR CODE HERE: Create similar plots for age and distance_to_city
# Plot 2: Age vs Price
axes[0, 1].______(______)
axes[0, 1].set_xlabel('______')
axes[0, 1].set_ylabel('______')
axes[0, 1].set_title('______')

# Plot 3: Distance to City vs Price
______

# Plot 4: School Rating vs Price
axes[1, 0].scatter(df['school_rating'], df['price'], alpha=0.5)
axes[1, 0].set_xlabel('School Rating')
axes[1, 0].set_ylabel('Price ($)')
axes[1, 0].set_title('Price vs School Rating')

# YOUR CODE HERE: Create box plots for categorical variables
# Plot 5: Price by number of bedrooms
df.boxplot(column='price', by='bedrooms', ax=axes[1, 1])
axes[1, 1].set_xlabel('Number of Bedrooms')
axes[1, 1].set_ylabel('Price ($)')
axes[1, 1].set_title('Price by Bedrooms')

# Plot 6: Price by pool availability
______

plt.tight_layout()
plt.show()

### Exercise 2.3: Correlation Analysis
**Task**: Analyze correlations between features and identify potential multicollinearity issues.

In [ ]:
# TODO: Create a correlation heatmap
plt.figure(figsize=(10, 8))

# YOUR CODE HERE: Calculate correlation matrix and create heatmap
correlation_matrix = ______
sns.heatmap(______, annot=True, cmap='coolwarm', center=0, fmt='.2f')

plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Identify highly correlated features
print("Highly correlated feature pairs (|correlation| > 0.7):")
# YOUR CODE HERE: Find and print highly correlated pairs
______

## Part 3: Data Splitting - The ML Approach

Following the machine learning approach, we'll split our data into training, validation, and test sets.

### Why Split Data?
- **Training set**: Used to fit the model
- **Validation set**: Used for model selection and hyperparameter tuning
- **Test set**: Used for final, unbiased evaluation

### Exercise 3.1: Create Train/Validation/Test Split

In [ ]:
# Prepare features and target
X = df.drop('price', axis=1)
y = df['price']

# TODO: Create a 60/20/20 train/validation/test split
# Step 1: First split into temp (80%) and test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=______, random_state=42
)

# Step 2: Split temp into train (75% of temp = 60% of total) and validation (25% of temp = 20% of total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=______, random_state=42
)

# Print the shapes
print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validation set: {X_val.shape[0]} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nFeatures: {X_train.shape[1]}")

## Part 4: Baseline Model - Simple Linear Regression

Let's start with a simple linear regression model as our baseline.

### Exercise 4.1: Train Baseline Model

In [ ]:
# TODO: Train a simple linear regression model
baseline_model = LinearRegression()

# YOUR CODE HERE: Fit the model on training data
______

# Make predictions on all three sets
y_pred_train = baseline_model.predict(X_train)
y_pred_val = ______  # YOUR CODE HERE
y_pred_test = ______  # YOUR CODE HERE

print("Baseline model trained!")

### Exercise 4.2: Evaluate Baseline Model
**Task**: Calculate R², RMSE, and MAE for all three sets to check for overfitting.

In [ ]:
def evaluate_model(y_true, y_pred, set_name):
    """
    Calculate and display evaluation metrics.
    """
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = ______  # YOUR CODE HERE: Calculate MAE
    
    print(f"{set_name} Set:")
    print(f"  R² Score: {r2:.4f}")
    print(f"  RMSE: ${rmse:,.2f}")
    print(f"  MAE: ${mae:,.2f}")
    return r2, rmse, mae

# Evaluate on all sets
print("Baseline Model Performance:")
print("="*40)
train_metrics = evaluate_model(y_train, y_pred_train, "Training")
val_metrics = evaluate_model(______) # YOUR CODE HERE
test_metrics = evaluate_model(______) # YOUR CODE HERE

# Check for overfitting
print("\nOverfitting Analysis:")
print(f"Train-Val R² Gap: {train_metrics[0] - val_metrics[0]:.4f}")
print(f"Train-Test R² Gap: {train_metrics[0] - test_metrics[0]:.4f}")

## Part 5: Feature Engineering

Let's improve our model by engineering new features.

### Exercise 5.1: Create Polynomial and Interaction Features

In [ ]:
# TODO: Add engineered features
def add_engineered_features(df):
    """
    Add polynomial and interaction features.
    """
    df_eng = df.copy()
    
    # Add polynomial features
    df_eng['sqft_squared'] = df_eng['sqft'] ** 2
    df_eng['age_squared'] = ______  # YOUR CODE HERE
    
    # Add interaction features
    df_eng['sqft_bedrooms'] = df_eng['sqft'] * df_eng['bedrooms']
    df_eng['sqft_bathrooms'] = ______  # YOUR CODE HERE
    df_eng['age_distance'] = ______  # YOUR CODE HERE: age * distance_to_city
    
    # Add ratio features
    df_eng['bath_bed_ratio'] = df_eng['bathrooms'] / (df_eng['bedrooms'] + 1)
    df_eng['sqft_per_bedroom'] = ______  # YOUR CODE HERE
    
    # Add derived features
    df_eng['luxury_score'] = (
        df_eng['has_pool'] + 
        df_eng['has_garage'] + 
        (df_eng['bathrooms'] > 2).astype(int)
    )
    
    return df_eng

# Apply feature engineering
X_train_eng = add_engineered_features(X_train)
X_val_eng = add_engineered_features(X_val)
X_test_eng = add_engineered_features(X_test)

print(f"Original features: {X_train.shape[1]}")
print(f"Engineered features: {X_train_eng.shape[1]}")
print(f"\nNew features added:")
new_features = set(X_train_eng.columns) - set(X_train.columns)
for feat in new_features:
    print(f"  - {feat}")

### Exercise 5.2: Feature Scaling
**Task**: Scale features to improve model performance.

In [ ]:
# TODO: Scale the features
scaler = StandardScaler()

# Fit on training data and transform all sets
X_train_scaled = scaler.fit_transform(X_train_eng)
X_val_scaled = ______  # YOUR CODE HERE: Only transform, don't fit!
X_test_scaled = ______  # YOUR CODE HERE: Only transform, don't fit!

print("Features scaled successfully!")
print(f"\nExample of scaling effect on 'sqft':")
print(f"  Original mean: {X_train_eng['sqft'].mean():.2f}")
print(f"  Original std: {X_train_eng['sqft'].std():.2f}")
print(f"  Scaled mean: {X_train_scaled[:, 0].mean():.2f}")
print(f"  Scaled std: {X_train_scaled[:, 0].std():.2f}")

## Part 6: Model Complexity and Overfitting

Let's explore different levels of model complexity and see how they affect performance.

### Exercise 6.1: Compare Different Polynomial Degrees

In [ ]:
# TODO: Test different polynomial degrees
degrees = [1, 2, 3, 4, 5]
train_scores = []
val_scores = []

for degree in degrees:
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train_scaled)
    X_val_poly = ______  # YOUR CODE HERE: Transform validation set
    
    # Train model
    model = LinearRegression()
    model.fit(______) # YOUR CODE HERE: Fit on polynomial training data
    
    # Evaluate
    train_score = model.score(X_train_poly, y_train)
    val_score = ______  # YOUR CODE HERE: Score on validation set
    
    train_scores.append(train_score)
    val_scores.append(val_score)
    
    print(f"Degree {degree}: Train R²={train_score:.4f}, Val R²={val_score:.4f}, "
          f"Gap={train_score-val_score:.4f}")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(degrees, train_scores, 'o-', label='Training R²', linewidth=2)
plt.plot(degrees, val_scores, 'o-', label='Validation R²', linewidth=2)
plt.xlabel('Polynomial Degree')
plt.ylabel('R² Score')
plt.title('Model Complexity vs Performance')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Part 7: Regularization

To combat overfitting, let's apply regularization techniques.

### Exercise 7.1: Ridge Regression (L2 Regularization)

In [ ]:
# TODO: Test different alpha values for Ridge regression
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
ridge_train_scores = []
ridge_val_scores = []

for alpha in alphas:
    # Train Ridge model
    ridge = Ridge(alpha=alpha)
    ridge.fit(______) # YOUR CODE HERE: Fit on scaled training data
    
    # Evaluate
    train_score = ridge.score(X_train_scaled, y_train)
    val_score = ______  # YOUR CODE HERE
    
    ridge_train_scores.append(train_score)
    ridge_val_scores.append(val_score)
    
    print(f"Alpha={alpha:7.3f}: Train R²={train_score:.4f}, Val R²={val_score:.4f}")

# Find best alpha
best_alpha_idx = np.argmax(ridge_val_scores)
best_alpha = alphas[best_alpha_idx]
print(f"\nBest alpha: {best_alpha} with validation R² = {ridge_val_scores[best_alpha_idx]:.4f}")

### Exercise 7.2: Lasso Regression (L1 Regularization)

In [ ]:
# TODO: Implement Lasso regression for feature selection
lasso = Lasso(alpha=10)
lasso.fit(______) # YOUR CODE HERE

# Check which features were selected (non-zero coefficients)
feature_names = X_train_eng.columns
lasso_coef = lasso.coef_

# Count non-zero coefficients
n_selected = ______  # YOUR CODE HERE: Count non-zero coefficients
print(f"Lasso selected {n_selected} out of {len(feature_names)} features")

# Display selected features
print("\nSelected features (non-zero coefficients):")
selected_features = [(name, coef) for name, coef in zip(feature_names, lasso_coef) if coef != 0]
selected_features.sort(key=lambda x: abs(x[1]), reverse=True)
for name, coef in selected_features[:10]:  # Show top 10
    print(f"  {name:20s}: {coef:10.2f}")

## Part 8: Cross-Validation

Let's use cross-validation for more robust model evaluation.

### Exercise 8.1: K-Fold Cross-Validation

In [ ]:
# TODO: Perform 5-fold cross-validation
from sklearn.model_selection import KFold

# Combine train and validation for cross-validation
X_train_val = np.vstack([X_train_scaled, X_val_scaled])
y_train_val = np.concatenate([y_train, y_val])

# Test different models with cross-validation
models = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(alpha=best_alpha),
    'Lasso': Lasso(alpha=10),
    'ElasticNet': ElasticNet(alpha=10, l1_ratio=0.5)
}

cv_results = {}
for name, model in models.items():
    # Perform cross-validation
    cv_scores = cross_val_score(
        model, X_train_val, y_train_val, 
        cv=______, # YOUR CODE HERE: Number of folds
        scoring='r2'
    )
    
    cv_results[name] = cv_scores
    print(f"{name:12s}: Mean R² = {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

# Visualize cross-validation results
plt.figure(figsize=(10, 6))
plt.boxplot(cv_results.values(), labels=cv_results.keys())
plt.ylabel('R² Score')
plt.title('Cross-Validation Results Comparison')
plt.grid(True, alpha=0.3)
plt.show()

## Part 9: Learning Curves

Learning curves help us understand if our model would benefit from more data.

### Exercise 9.1: Generate Learning Curves

In [ ]:
# TODO: Generate learning curves
train_sizes, train_scores, val_scores = learning_curve(
    Ridge(alpha=best_alpha), 
    X_train_val, 
    y_train_val,
    cv=5,
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='r2'
)

# Calculate mean and std
train_mean = ______  # YOUR CODE HERE: Calculate mean across CV folds
train_std = ______   # YOUR CODE HERE: Calculate std across CV folds
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', label='Training Score', linewidth=2)
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.plot(train_sizes, val_mean, 'o-', label='Validation Score', linewidth=2)
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('R² Score')
plt.title('Learning Curves - Ridge Regression')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.show()

# Interpret the curves
gap = train_mean[-1] - val_mean[-1]
print(f"Final training score: {train_mean[-1]:.4f}")
print(f"Final validation score: {val_mean[-1]:.4f}")
print(f"Gap: {gap:.4f}")

if gap > 0.1:
    print("\nModel shows signs of overfitting. Consider:")
    print("- Increasing regularization")
    print("- Reducing model complexity")
    print("- Adding more training data")
elif val_mean[-1] < 0.7:
    print("\nModel shows signs of underfitting. Consider:")
    print("- Adding more features")
    print("- Increasing model complexity")
    print("- Reducing regularization")
else:
    print("\nModel appears well-fitted!")

## Part 10: Production Pipeline

Let's create a production-ready pipeline that combines all preprocessing and modeling steps.

### Exercise 10.1: Build Complete Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# TODO: Create a complete pipeline
# Define numeric features
numeric_features = X_train.columns.tolist()

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ]
)

# Create full pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', ______),  # YOUR CODE HERE: Add polynomial features (degree=2)
    ('regressor', ______)  # YOUR CODE HERE: Add Ridge regression with best alpha
])

# Fit pipeline on training data
pipeline.fit(X_train, y_train)

# Evaluate on all sets
print("Pipeline Performance:")
print("="*40)
print(f"Training R²: {pipeline.score(X_train, y_train):.4f}")
print(f"Validation R²: {pipeline.score(X_val, y_val):.4f}")
print(f"Test R²: {pipeline.score(X_test, y_test):.4f}")

## Part 11: Final Model Evaluation and Business Insights

### Exercise 11.1: Final Test Set Evaluation

In [ ]:
# Make final predictions on test set
y_test_pred = pipeline.predict(X_test)

# Calculate final metrics
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)

print("FINAL MODEL PERFORMANCE ON TEST SET")
print("="*40)
print(f"R² Score: {test_r2:.4f}")
print(f"RMSE: ${test_rmse:,.2f}")
print(f"MAE: ${test_mae:,.2f}")
print(f"\nThis means:")
print(f"- The model explains {test_r2*100:.1f}% of price variance")
print(f"- Average prediction error is ${test_mae:,.0f}")
print(f"- 68% of predictions are within ${test_rmse:,.0f} of actual price")

### Exercise 11.2: Residual Analysis

In [ ]:
# TODO: Analyze residuals
residuals = ______  # YOUR CODE HERE: Calculate residuals (actual - predicted)

# Create residual plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Residuals vs Predicted
axes[0, 0].scatter(y_test_pred, residuals, alpha=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Predicted Price')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residuals vs Predicted Values')

# Plot 2: Histogram of residuals
axes[0, 1].hist(residuals, bins=30, edgecolor='black')
axes[0, 1].set_xlabel('Residuals')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Residuals')

# Plot 3: Q-Q plot
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot')

# Plot 4: Actual vs Predicted
axes[1, 1].scatter(y_test, y_test_pred, alpha=0.5)
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 1].set_xlabel('Actual Price')
axes[1, 1].set_ylabel('Predicted Price')
axes[1, 1].set_title('Actual vs Predicted Prices')

plt.tight_layout()
plt.show()

### Exercise 11.3: Feature Importance for Business Insights

In [ ]:
# Get feature importance from the final model
# Note: This is simplified - in production you'd need to account for scaling
final_model = Ridge(alpha=best_alpha)
final_model.fit(X_train_scaled, y_train)

# Get coefficients
feature_importance = pd.DataFrame({
    'feature': X_train_eng.columns,
    'coefficient': final_model.coef_,
    'abs_coefficient': np.abs(final_model.coef_)
}).sort_values('abs_coefficient', ascending=False)

# Display top features
print("Top 10 Most Important Features:")
print("="*50)
for idx, row in feature_importance.head(10).iterrows():
    direction = "increases" if row['coefficient'] > 0 else "decreases"
    print(f"{row['feature']:20s}: {direction} price")

# Visualize feature importance
plt.figure(figsize=(10, 6))
top_features = feature_importance.head(10)
plt.barh(range(len(top_features)), top_features['abs_coefficient'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Absolute Coefficient Value')
plt.title('Top 10 Feature Importance')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Part 12: Business Recommendations

Based on our analysis, let's provide actionable insights for Sunshine Realty.

### Exercise 12.1: Generate Business Report

In [ ]:
print("BUSINESS INSIGHTS REPORT FOR SUNSHINE REALTY")
print("="*60)

print("\n1. MODEL PERFORMANCE")
print("-"*40)
print(f"   • Accuracy: The model can predict house prices with an average")
print(f"     error of ${test_mae:,.0f}, which is {test_mae/y_test.mean()*100:.1f}% of the average price.")
print(f"   • Confidence: {test_r2*100:.1f}% of price variation is explained by the model.")

print("\n2. KEY VALUE DRIVERS")
print("-"*40)
print("   Top factors that increase property value:")
for idx, row in feature_importance[feature_importance['coefficient'] > 0].head(5).iterrows():
    print(f"   • {row['feature']}")

print("\n3. PRICING RECOMMENDATIONS")
print("-"*40)
print("   • Properties with pools command a premium")
print("   • School quality significantly impacts price")
print("   • Square footage remains the strongest predictor")

print("\n4. INVESTMENT INSIGHTS")
print("-"*40)
# Calculate some specific insights
pool_premium = final_model.coef_[list(X_train_eng.columns).index('has_pool')] * scaler.scale_[list(X_train_eng.columns).index('has_pool')]
print(f"   • Adding a pool could increase value by approximately ${abs(pool_premium):,.0f}")
print(f"   • Properties closer to city center have higher values")
print(f"   • Newer properties maintain value better")

print("\n5. NEXT STEPS")
print("-"*40)
print("   • Deploy model for real-time price predictions")
print("   • Monitor model performance monthly")
print("   • Retrain quarterly with new sales data")
print("   • Consider adding neighborhood-specific features")

## Conclusion

Congratulations! You've successfully completed a comprehensive linear regression analysis for real estate price prediction. 

### Key Takeaways:

1. **ML Approach**: We focused on prediction accuracy rather than statistical inference
2. **Data Splitting**: Used train/validation/test splits to avoid overfitting
3. **Feature Engineering**: Created polynomial and interaction features to capture complex relationships
4. **Regularization**: Applied Ridge and Lasso to prevent overfitting
5. **Model Selection**: Used validation set and cross-validation for unbiased model selection
6. **Production Pipeline**: Built a complete pipeline ready for deployment
7. **Business Value**: Translated technical results into actionable business insights

### Skills Practiced:
- Data exploration and visualization
- Feature engineering and scaling
- Model training and evaluation
- Overfitting detection and prevention
- Cross-validation and hyperparameter tuning
- Pipeline creation for production
- Business communication of technical results